In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pymc as pm
import arviz as az
from jax import numpy as jnp

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm

from inputs.constants import ANALYSIS_START_DATE, ANALYSIS_END_DATE, PLOT_START_DATE, PROJECT_PATH, SUPPLEMENT_PATH
from autumn.infrastructure.remote import springboard
from autumn.core.runs import ManagedRun
from aust_covid.inputs import load_calibration_targets, load_who_data, load_serosurvey_data, get_ifrs, load_raw_pop_data, get_raw_state_mobility
from aust_covid.model import build_model
from aust_covid.plotting import plot_key_outputs, plot_cdr_examples, plot_subvariant_props, plot_dispersion_examples
from aust_covid.plotting import plot_state_mobility, plot_processed_mobility, plot_example_model_matrices
from aust_covid.calibration import get_priors, get_targets
from emutools.tex import StandardTexDoc
from emutools.inputs import load_param_info
from emutools.calibration import param_table_to_tex, round_sigfig, sample_idata, get_sampled_outputs, plot_param_progression, view_posterior_comparison, tabulate_priors, tabulate_param_results

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'supplement', "Australia's 2023 Omicron Waves Supplement", 'austcovid')

In [ ]:
param_info = load_param_info(PROJECT_PATH / 'inputs' / 'parameters.yml')
ifrs = get_ifrs(app_doc)
param_info['value'].update(ifrs)
parameters = param_info['value'].to_dict()

In [ ]:
aust_model = build_model(app_doc, mobility_sens=True)

In [ ]:
priors = get_priors()
prior_names = [p.name for p in priors]

In [ ]:
targets = get_targets(app_doc)

In [ ]:
calibration_model = BayesianCompartmentalModel(aust_model, parameters, priors, targets)

In [ ]:
# with pm.Model() as pmc_model:
#     start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in parameters.items() if k in calibration_model.priors}
#     variables = epm.use_model(calibration_model)
#     map_params = pm.find_MAP(start=start_params, vars=variables, include_transformed=False)
#     map_params = {k: float(v) for k, v in map_params.items()}
# print('Best calibration parameters found:')
# for i_param, param in enumerate([p for p in map_params if '_dispersion' not in p]):
#     print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')
# parameters.update(map_params)
# aust_model.run(parameters=parameters)

In [ ]:
app_doc.include_table(param_table_to_tex(param_info, prior_names), section='Parameters', col_splits=[0.17, 0.15, 0.15, 0.53], longtable=True)
app_doc.include_table(tabulate_priors(priors, param_info), section='Calibration', col_splits=[0.25] * 4)

In [ ]:
with pm.Model() as pm_model:
    variables = epm.use_model(calibration_model)
    idata_local = pm.sample(step=[pm.DEMetropolis(variables)], draws=100, tune=0, cores=9, chains=18, progressbar=True)
idata_local.to_netcdf('calibration_out.nc')

In [ ]:
def run_calibration(bridge: springboard.task.TaskBridge, calibration_model: BayesianCompartmentalModel):
    import multiprocessing as mp
    mp.set_start_method('forkserver')
    
    with pm.Model() as pm_model:
        variables = epm.use_model(calibration_model)
        idata_raw = pm.sample(step=[pm.DEMetropolisZ(variables)], draws=10000, tune=2000, cores=8, chains=8, progressbar=False)
    
    idata_raw.to_netcdf(str(bridge.out_path / 'calibration_out.nc'))
    bridge.logger.info('Calibration complete')
    
mspec = springboard.EC2MachineSpec(8, 2, 'compute')
tspec = springboard.TaskSpec(run_calibration, {'calibration_model': calibration_model})
run_path = springboard.launch.get_autumn_project_run_path('aust_covid', 'base_case_analysis', 'try_DEMZ_10k')
run_path

In [ ]:
aust_covid_commands = [
    'git clone --branch main https://github.com/monash-emu/aust-covid',
    'pip install -e ./aust-covid',
]
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path, branch=None, extra_commands=aust_covid_commands)